In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [26]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://job-boards.greenhouse.io/ziprecruiter/jobs/6165341?gh_src=89c6ee071us")
page_data = loader.load().pop().page_content
print(page_data)

Job Application for Software Engineer - New Grad at ZipRecruiterBack to jobsSoftware Engineer - New GradSanta Monica, CAApplyWe offer a hybrid work environment. Most US-based positions can also be performed remotely (any exceptions will be noted in the Minimum Qualifications below.)Our Mission: 
To actively connect people to their next great opportunity. 
Who We Are: 
ZipRecruiter is the fastest-growing online employment marketplace for both desktop and mobile traffic. Powered by AI-driven smart matching technology, we’re building connections for millions of job seekers and businesses of all sizes through innovative mobile, web, and email services, as well as through partnerships with the best job boards on the web. As the #1 rated job search app on iOS & Android for the past 6 years, our customers are our priority. 
About the Team:
We’re looking for software engineers who are passionate about building an intuitive experience for our marketplace serving millions of jobseekers and tens 

In [27]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)
print(res.content)

```json
{
  "role": "Software Engineer - New Grad",
  "experience": "New Grad or recent graduate with a graduation date from Spring 2023 to Spring 2024",
  "skills": [
    "Fluency with at least 1 general purpose programming language (e.g., Python, Java, Go, C++, Javascript)",
    "Interest in becoming more specialized in at least one of the following disciplines: Big Data, Full Stack, Machine Learning, Mobile - iOS, Mobile - Android",
    "Fluency in multiple programming languages or ability to pick up new languages quickly",
    "Experience working with both non-relational and relational database systems",
    "Exposure to developing client side interactions using frameworks like React along with Vanilla Javascript, CSS, and HTML",
    "Analytical and problem-solving skills along with a self-starter mentality",
    "Passion for and experience with creating an intuitive, user-friendly experience"
  ],
  "description": "Take part in the full life cycle of user-facing applications at sc

In [28]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer - New Grad',
 'experience': 'New Grad or recent graduate with a graduation date from Spring 2023 to Spring 2024',
 'skills': ['Fluency with at least 1 general purpose programming language (e.g., Python, Java, Go, C++, Javascript)',
  'Interest in becoming more specialized in at least one of the following disciplines: Big Data, Full Stack, Machine Learning, Mobile - iOS, Mobile - Android',
  'Fluency in multiple programming languages or ability to pick up new languages quickly',
  'Experience working with both non-relational and relational database systems',
  'Exposure to developing client side interactions using frameworks like React along with Vanilla Javascript, CSS, and HTML',
  'Analytical and problem-solving skills along with a self-starter mentality',
  'Passion for and experience with creating an intuitive, user-friendly experience'],
 'description': 'Take part in the full life cycle of user-facing applications at scale, from design to implementation

In [32]:
type(json_res)

dict

In [33]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,React,https://linktr.ee/bhumikadasari
1,"Angular, Express, Node.js, MongoDB",https://bookthrift.surge.sh/#!/
2,"Flask, SQLAlchemy, Jinja2, Pillow, HTML/CSS/Bo...",https://github.com/dasaribhumika/SSN-Blog
3,"Streamlit, Pandas, Scikit-learn, Matplotlib, S...",https://github.com/dasaribhumika/e-learning-pl...
4,"Flask, Javascript, Postman",https://github.com/dasaribhumika/House-Price-P...
5,"React, Javascript",https://bhumika-dasari-portfolio.netlify.app/
6,"Streamit, Mistral, LLM, GenAI, NLP",https://github.com/dasaribhumika/sage-health-c...
7,"Python, Machine Learning, LSTM, CNN, RNN, NLTK...",https://github.com/dasaribhumika/NLP_Financial...
8,PowerBI,https://github.com/dasaribhumika/Zomato-Analys...


In [36]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [45]:
# Access the job details
role = json_res.get('role')
experience = json_res.get('experience')
skills = json_res.get('skills')
description = json_res.get('description')

# Print the extracted information
print(f"Role: {role}")
print(f"Experience: {experience}")
print(f"Skills: {skills}")
print(f"Description: {description}")

# If you need to perform operations based on the skills, you can do so like this:
for skill in skills:
    links = collection.query(query_texts=skill, n_results=2).get('metadatas', [])
    print(f"Links for skill '{skill}': {links}")



Role: Software Engineer - New Grad
Experience: New Grad or recent graduate with a graduation date from Spring 2023 to Spring 2024
Skills: ['Fluency with at least 1 general purpose programming language (e.g., Python, Java, Go, C++, Javascript)', 'Interest in becoming more specialized in at least one of the following disciplines: Big Data, Full Stack, Machine Learning, Mobile - iOS, Mobile - Android', 'Fluency in multiple programming languages or ability to pick up new languages quickly', 'Experience working with both non-relational and relational database systems', 'Exposure to developing client side interactions using frameworks like React along with Vanilla Javascript, CSS, and HTML', 'Analytical and problem-solving skills along with a self-starter mentality', 'Passion for and experience with creating an intuitive, user-friendly experience']
Description: Take part in the full life cycle of user-facing applications at scale, from design to implementation and testing to production. Deve

In [50]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Bhumika Dasari, a tech enthusiast and Master's student in Computer Science at Syracuse University. 
        You are passionate about AI/GenAI and have experience as a Software Developer. Currently, you are working as a Graduate Research Assistant, 
        where you are involved in developing digital literacy workshops and analyzing data to empower underserved communities. 
        Your goal is to write a cold email to potential clients or employers regarding your interest in opportunities within AI/GenAI and 
        software development roles. Describe your capabilities and how your skills can fulfill their needs.
        Include relevant projects and experiences that showcase your expertise in software development and AI, emphasizing your commitment 
        to deliver innovative solutions and your enthusiasm for technology.
        Also add the most relevant ones from the following links to showcase my portfolio: {link_list}
        Remember to maintain a professional tone and highlight your unique contributions as a tech enthusiast.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Exploring Opportunities in AI/GenAI and Software Development

Dear Hiring Manager,

I am excited to introduce myself as a highly motivated and detail-oriented Master's student in Computer Science at Syracuse University, with a strong passion for Artificial Intelligence (AI) and Generative AI (GenAI). As a tech enthusiast with experience as a Software Developer and a Graduate Research Assistant, I am confident in my ability to deliver innovative solutions that drive business growth and empower underserved communities.

With a solid foundation in software development, I have honed my skills in programming languages such as Python, Java, and C++. My experience in developing digital literacy workshops has not only deepened my understanding of AI and GenAI but also instilled in me a commitment to creating accessible and inclusive technology. As a researcher, I have analyzed data to identify trends and patterns, which has allowed me to develop data-driven solutions that cater to the